In [2]:
print("hii nivash")

hii nivash


#froward pass

In [3]:
import math

class value:
    def __init__(self, data, children = (), op = ''):
        self.data = data
        self.op = op
        self.prev = set(children)
        self._backward = lambda : None
        self.grad = 0.0
        
        
    def __repr__(self):
        # return f"value is :{self.data}  gradient is : {self.grad} operation is :  {self.op} previous values: {self.prev}\n backward function : {self.backward}\n"
        return f"value(data = {self.data} grad = {self.grad})"
    
    def __add__(self, other):
        if not isinstance(other, value):
            other = value(other)
            
            
        result = value(self.data + other.data, (self, other), '+')
        def grad_calculator():
            self.grad += result.grad
            other.grad += result.grad
        result._backward = grad_calculator 
        return result
    
    # def __radd__(self, other):
    #     return self + other
    
    
    def __neg__(self):
        
        return self * -1
    
    
    def __sub__(self, other):
        return self + (-other)
        
        
    def __pow__(self, other):
            
        assert isinstance(other, int | float)
            
        result = value(self.data**other, (self,), f'{other}')
        def grad_calculator():
            self.grad += other * (self.data** (other - 1)) * result.grad
            
        result._backward = grad_calculator
        return result
        
    def __truediv__(self, other):
        return self * (other ** -1)
        
        
    
    def __mul__(self, other):
        if not isinstance(other, value):
            other = value(other)
            
        result = value(self.data * other.data, (self, other) , '*')
        def grad_calculator():
            self.grad += result.grad * other.data
            other.grad += result.grad * self.data
        result._backward = grad_calculator
        
        return result
    
    # def __rmul__(self, other):
    #     return self * other
    
    def exp(self):
        result = value(math.exp(self.data), (self,), 'exp')
        def grad_calculator():
            self.grad += result.grad * math.exp(self.data)
        result._backward = grad_calculator
        return result
    
    
    def tanh(self):
        n = self.data
        out = (math.exp(2.0*n) - 1)/(math.exp(2.0*n) + 1)
        result = value(out, (self,), 'tanh')
        def grad_calculator():
            self.grad += (1 - out**2) * result.grad
            
            
        result._backward = grad_calculator
            
        
        return result
    
    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v.prev:
                    build_topo(child)
                topo.append(v)
            
        build_topo(self)
        topo.reverse()
        self.grad = 1.0
        
        
        for x in topo:
            x._backward()
            
    __radd__ = __add__
    __rmul__ = __mul__
            
        
                    

In [4]:
x = value(3.0)
y = value(4.0)


print(y*3, 4.5*y, x**4,  x/2, 3 + x, x + 9)

value(data = 12.0 grad = 0.0) value(data = 18.0 grad = 0.0) value(data = 81.0 grad = 0.0) value(data = 1.5 grad = 0.0) value(data = 6.0 grad = 0.0) value(data = 12.0 grad = 0.0)


#direct tanh usage

In [5]:
a = value(0.4)
b= value(0.8)
e= a*b
c = value(1.0)
d = c+e
f = value(-1.5)
l = d*f
o = l.tanh()

o.backward()

print("a is : " ,a, "\nb is : ",b, "\nc is :",c, "\nd is : ",d, "\ne is : ",e, "\nf is : ",f,"\nl is : ",l, "\no is : ", o )



a is :  value(data = 0.4 grad = -0.08811156691047467) 
b is :  value(data = 0.8 grad = -0.044055783455237335) 
c is : value(data = 1.0 grad = -0.11013945863809332) 
d is :  value(data = 1.32 grad = -0.11013945863809332) 
e is :  value(data = 0.32000000000000006 grad = -0.11013945863809332) 
f is :  value(data = -1.5 grad = 0.09692272360152213) 
l is :  value(data = -1.98 grad = 0.07342630575872888) 
o is :  value(data = -0.9625869800912908 grad = 1.0)


#indirect tanh usage using chunck functions to check whether the chunking is done correctly or not

In [6]:
a = value(0.4)
b= value(0.8)
e= a*b
c = value(1.0)
d = c+e
f = value(-1.5)
l = d*f

m = 2*l
n = m.exp()

o = (n-1)/(n+1)

o.backward()

print("a is : " ,a, "\nb is : ",b, "\nc is :",c, "\nd is : ",d, "\ne is : ",e, "\nf is : ",f,"\nl is : ",l, "\no is : ", o )


a is :  value(data = 0.4 grad = -0.08811156691047475) 
b is :  value(data = 0.8 grad = -0.04405578345523738) 
c is : value(data = 1.0 grad = -0.11013945863809343) 
d is :  value(data = 1.32 grad = -0.11013945863809343) 
e is :  value(data = 0.32000000000000006 grad = -0.11013945863809343) 
f is :  value(data = -1.5 grad = 0.09692272360152222) 
l is :  value(data = -1.98 grad = 0.07342630575872895) 
o is :  value(data = -0.9625869800912908 grad = 1.0)


#using pytorch to implement the same thing

In [48]:
import torch as t

a = t.tensor(0.4, requires_grad = True)
b= t.tensor(0.8, requires_grad = True)
e= a*b
c = t.tensor(1.0, requires_grad = True)
d = c+e
f = t.tensor(-1.5, requires_grad = True)
l = d*f



m = 2*l
n = m.exp()


o = (n-1)/(n+1)

d.retain_grad()
e.retain_grad()
l.retain_grad()
o.retain_grad()
o.backward()


# print("a : " , a.item(), a.grad , "\nb is : ",b.item() , b.grad,  "\nc is :",c, "\nd is : ",d, "\ne is : ",e, "\nf is : ",f,"\nl is : ",l, "\no is : ", o )

mydict = {'a': a , 'b': b, 'c': c, 'd': d, 'e' : e , 'f' : f, 'l' : l, 'o' : o}


for key in mydict:
    print(f"{key} is : {mydict[key]} , {key}.grad is : {mydict[key].grad} ")



a is : 0.4000000059604645 , a.grad is : -0.08811157196760178 
b is : 0.800000011920929 , b.grad is : -0.04405578598380089 
c is : 1.0 , c.grad is : -0.11013945937156677 
d is : 1.3200000524520874 , d.grad is : -0.11013945937156677 
e is : 0.320000022649765 , e.grad is : -0.11013945937156677 
f is : -1.5 , f.grad is : 0.09692272543907166 
l is : -1.9800000190734863 , l.grad is : 0.07342630624771118 
o is : -0.9625869989395142 , o.grad is : 1.0 
